In [ ]:
from qiskit.circuit.random import random_circuit
import qiskit
import random
import numpy as np
from qiskit import QuantumCircuit, execute, Aer, IBMQ, QuantumRegister, ClassicalRegister
from qiskit.visualization import plot_histogram, plot_gate_map, plot_circuit_layout
from math import sqrt, pi
import numpy as np
from qiskit import *
from qiskit.tools.monitor import job_monitor
from qiskit.providers.aer.noise import NoiseModel
import matplotlib.pyplot as plt
%matplotlib inline
import pickle
from scipy.stats import mode
import numpy as np
import math
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Conv1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from tensorflow.keras.optimizers import SGD, Adam, Nadam
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.callbacks import LearningRateScheduler, EarlyStopping,ModelCheckpoint
from tensorflow.keras.models import model_from_json
import numpy as np
import matplotlib.pyplot as plt
from sympy import fwht,ifwht
import pickle
from scipy.optimize import curve_fit
from sklearn.linear_model import LinearRegression
from sklearn.utils import shuffle
from scipy.spatial import distance
import timeit
from scipy.linalg import hadamard
import pandas as pd

In [ ]:
dbfile = open('AllInputsData/BelemAll', 'rb');
Data = pickle.load(dbfile);
dbfile.close()
Data.shape

In [ ]:
D=np.mean(Data,axis=1)
D.shape

In [ ]:
from math import factorial as fac
def nCr(n,r):
    return fac(n) // fac(r) // fac(n-r)

In [ ]:
def projection_simplex_sort(v, z=1):
    n_features = v.shape[0]
    u = np.sort(v)[::-1]
    cssv = np.cumsum(u) - z
    ind = np.arange(n_features) + 1
    cond = u - cssv / ind > 0
    rho = ind[cond][-1]
    theta = cssv[cond][-1] / float(rho)
    w = np.maximum(v - theta, 0)
    return w

In [ ]:
def model(x,A,alpha,B):
    return A*alpha**x+B
def remove_outliers(M,threshold):
    M_new=[]
    nCliffs_new=[]
    fit=curve_fit(model,nCliffs,M[:,0],p0=(0.9,0.9,0.9),bounds=(0,[1,1,1]),maxfev=2000)
    Aout,alphaout,Bout=fit[0]
    for i in range(M.shape[0]):
        actual=M[i,0]
        fitted=Aout*alphaout**nCliffs[i]+Bout 
        if (actual-fitted)**2<threshold:
            M_new.append(M[i,:])
            nCliffs_new.append(nCliffs[i])
    M_new=np.array(M_new)
    nCliffs_new=np.array(nCliffs_new)
    
    return M_new, nCliffs_new, Aout, alphaout, Bout

In [ ]:
def SetUp(Data, m_tr, nQ):
    #Split raw Data into Tr and Te
    lengths_tr = np.arange(0,m_tr)
    lengths_te = np.arange(m_tr,D.shape[0])
    Data_tr = Data[lengths_tr]
    Data_te = Data[lengths_te]
    
    #Initialize data arrays
    size_tr = nCr(m_tr,2)
    size_te = nCr(D.shape[0]-m_tr,2)
    X_tr = np.zeros([size_tr,2**nQ+1], dtype = np.float32);
    Y_tr = np.zeros([size_tr,2**nQ], dtype = np.float32);
    X_te = np.zeros([size_te,2**nQ+1], dtype = np.float32);
    Y_te = np.zeros([size_te,2**nQ], dtype = np.float32);
    
    #Fill data arrays
    #Training
    a = 0;
    b = 0;
    bb = 0;
    for b in range(1,len(lengths_tr)):
        if (b!=1):
            bb = bb + b - 1;
        for a in range(0,b):
            del_m = lengths_tr[b] - lengths_tr[a];
            B = Data_tr[b,:];
            A = Data_tr[a,:];
            X_tr[(bb+a),:2**nQ] = A;
            X_tr[(bb+a),2**nQ] = del_m;
            Y_tr[(bb+a),:] = B;
        
    #Testing
    a = 0;
    b = 0;
    bb = 0;
    for b in range(1,len(lengths_te)):
        if (b!=1):
            bb = bb + b - 1;
        for a in range(0,b):
            del_m = lengths_te[b] - lengths_te[a];
            B = Data_te[b,:];
            A = Data_te[a,:];
            X_te[(bb+a),:2**nQ] = A;
            X_te[(bb+a),2**nQ] = del_m;
            Y_te[(bb+a),:] = B;
    
    #DONE
    return (X_tr,Y_tr,X_te,Y_te)

In [ ]:
def scheduler(epoch, lr):
  if epoch < 15:
    return lr
  else:
    return lr * math.exp(-0.1)

In [ ]:
import keras as K
def JSD(y_true, y_pred):
    M = 0.5*(y_true+y_pred);
    return (0.5*K.losses.kullback_leibler_divergence(y_true,M) + 0.5*K.losses.kullback_leibler_divergence(y_pred,M))

In [ ]:
def NN(X_tr,Y_tr,X_te,Y_te):
    #Setup Data
    X_train, Y_train = shuffle(X_tr, Y_tr)
    
    #Setup model
    model_A = Sequential()

    model_A.add(Dense(40, activation = 'relu' ,input_dim = 2**nQ + 1))
    model_A.add(Dense(38, activation='relu'))
    model_A.add(Dense(34, activation='relu'))
    model_A.add(Dense(2**nQ, activation='softmax')) 

    # Compile model
    #epochs = 10
    learning_rate = 0.0001
    #decay_rate = learning_rate / epochs
    #momentum = 0.01
    adam = Adam(lr=learning_rate,beta_1=0.9, beta_2=0.999, epsilon=1e-07)

    model_A.compile(loss = JSD,
                  optimizer = 'adam')

    #Learning Rate Scheduler Callback
    lrs = LearningRateScheduler(scheduler)

    # train model
    history_A = model_A.fit(X_train, Y_train, batch_size= 50, epochs=50,validation_data=(X_te, Y_te),verbose = 0)
    
    return(model_A, history_A)

In [ ]:
def CNN(X_tr,Y_tr,X_te,Y_te):
    #Setup Data
    X_train, Y_train = shuffle(X_tr, Y_tr)
    
    #Setup model
    model_A = Sequential()
    
    model_A.add(Conv1D(64, kernel_size=3, activation='relu', input_shape =(2**nQ+1,1)))
    model_A.add(Dense(40, activation = 'relu'))
    model_A.add(Dense(38, activation='relu'))
    model_A.add(Dense(34, activation='relu'))
    model_A.add(Dense(2**nQ, activation='softmax')) 

    # Compile model
    #epochs = 10
    learning_rate = 0.0001
    #decay_rate = learning_rate / epochs
    #momentum = 0.01
    adam = Adam(lr=learning_rate,beta_1=0.9, beta_2=0.999, epsilon=1e-07)

    model_A.compile(loss = JSD,
                  optimizer = 'adam')

    #Learning Rate Scheduler Callback
    lrs = LearningRateScheduler(scheduler)

    # train model
    history_A = model_A.fit(X_train, Y_train, batch_size= 50, epochs=30,callbacks = [lrs],validation_data=(X_te, Y_te),verbose = 0)
    
    return(model_A, history_A)

In [ ]:
def LC(H):
    plt.plot(H.history['loss'])
    plt.title('Training loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train'], loc='upper left')
    plt.show()
    plt.plot(H.history['val_loss'])
    plt.title('Validation loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['test'], loc='upper left')
    plt.show()

In [ ]:
def Jump(Data,m_x,NN):
    x = np.zeros([1,2**nQ+1], dtype = np.float32);
    x[0,:2**nQ] = Data[m_x-1];
    J = []
    for i in range(D.shape[0]-m_x):
        m = m_x+i;
        true = Data[m];
        x[0,2**nQ] = m+1-m_x
        pred = NN.predict(x);
        jsd = JSD(true,pred)[0];
        J.append(jsd);
    return (J)

In [ ]:
def JumpPred(Data,m_x,NN):
    x = np.zeros([1,2**nQ+1], dtype = np.float32);
    x[0,:2**nQ] = Data[m_x-1];
    Pred = []
    for i in range(D.shape[0]-m_x):
        m = m_x+i;
        true = Data[m];
        x[0,2**nQ] = m+1-m_x
        pred = NN.predict(x);
        Pred.append(pred);
    return (Pred)

In [ ]:
Data = D
nQ = 5;
#Initialize data arrays
size = nCr(6,2)*(2**nQ)
comb = nCr(6,2)
lengths_tr = [1,20,40,60,80,100]

X_tr = np.zeros([size,2**nQ+1], dtype = np.float32);
Y_tr = np.zeros([size,2**nQ], dtype = np.float32);
#X_te = np.zeros([size,2**nQ+1], dtype = np.float32);
#Y_te = np.zeros([size,2**nQ], dtype = np.float32);

#Fill data arrays
#Training
for i in range(32):
    a = 0;
    b = 0;
    bb = 0;
    Data_tr = Data[:,i,:]
    for b in range(6):
        if (b!=1):
            bb = bb + b - 1;
        for a in range(0,b):
            del_m = lengths_tr[b] - lengths_tr[a];
            B = Data[b,i,:];
            A = Data[a,i,:];
            X_tr[(bb+a)+i*comb,:2**nQ] = A;
            X_tr[(bb+a)+i*comb,2**nQ] = del_m;
            Y_tr[(bb+a)+i*comb,:] = B;

In [ ]:
#X_tr,Y_tr,X_te,Y_te = SetUp(D, m_tr, nQ)

#X_tr.reshape(X_tr.shape[0],X_tr.shape[1],1)
#X_te.reshape(X_te.shape[0],X_te.shape[1],1)

N, H = NN(X_tr,Y_tr,X_tr,Y_tr);
print("Training Evaluation: ")
N.evaluate(X_tr, Y_tr, verbose=1)
print("Testing Evaluation: ")
N.evaluate(X_tr, Y_tr, verbose=1)

LC(H)